# 8 WorkFlow  Semillerio
## Este es un archivo base que incluye ajuste por "inflacion" -> usando ranking con cero fijo
El codigo esta copiado del repositorio _senior_ como mencionó Natalia en Zulip.
** Está hecho de esta forma para usar en los tres experimentos a ensamblar. **
Tomo como referencia el _semillerio_ al que le agrego el ajuste por inflación.


### 8.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 8.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



*  Si usted eligió modalidad *Gerencial* entonces NO debe modificar nada, su liderazgo logró que otras trabajaron por usted
*  Si usted eligió modalidad de *Analista Junior*, entonces donde dice archivo="gerencial_competencia_2025.csv.gz"  lo debe cambiar por  archivo="analistajr_competencia_2025.csv.gz"
*  Si usted eligió modalidad *Analista Senior*,  ya se las ingenierá SIN preguntar !

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/labo1r"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 8.3  Workflow Semillerio

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Jul 20 11:00:38 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2465555,131.7,5960914,318.4,5960914,318.4
Vcells,4968034,38.0,93800653,715.7,117174014,894.0


In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

### Usted DEBE ambiar "PARAM experimento" cada vez que desee correr un nuevo experimento

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 888877

PARAM$experimento <- "completo-std-77b"
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 8.3.1   Preprocesamiento del dataset

#### 8.3.1.1  DT incorporar dataset

In [ ]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

#### 8.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA a los valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos. - Lo dejamos asi porque no se notó diferencia significativa en los experimentos colaborativos.

In [ ]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 8.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [ ]:
# El cero se transforma en cero
# los positivos se rankean por su lado
# los negativos se rankean por su lado

drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}

In [ ]:
# por como armé los nombres de campos,
#  estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios

[1] "mrentabilidad"           "mrentabilidad_annual"   
 [3] "mcomisiones"             "mactivos_margen"        
 [5] "mpasivos_margen"         "mcuenta_corriente"      
 [7] "mcaja_ahorro"            "mcuentas_saldo"         
 [9] "mtarjeta_visa_consumo"   "mtarjeta_master_consumo"
[11] "mprestamos_personales"   "mpayroll"               
[13] "Master_mpagominimo"      "Visa_mpagominimo"

In [ ]:
# ejecuto el Data Drifting
setorder( dataset, numero_de_cliente, foto_mes )


PARAM$DR$metodo <- "rank_cero_fijo"

switch(PARAM$DR$metodo,
  "ninguno"        = cat("No hay correccion del data drifting"),
  "rank_simple"    = drift_rank_simple(campos_monetarios),
  "rank_cero_fijo" = drift_rank_cero_fijo(campos_monetarios),
  "deflacion"      = drift_deflacion(campos_monetarios),
  "dolar_blue"     = drift_dolarblue(campos_monetarios),
  "dolar_oficial"  = drift_dolaroficial(campos_monetarios),
  "UVA"            = drift_UVA(campos_monetarios),
  "estandarizar"   = drift_estandarizar(campos_monetarios)
)


inicio drift_rank_cero_fijo()
mrentabilidad  mrentabilidad_annual  mcomisiones  mactivos_margen  mpasivos_margen  mcuenta_corriente  mcaja_ahorro  mcuentas_saldo  mtarjeta_visa_consumo  mtarjeta_master_consumo  mprestamos_personales  mpayroll  Master_mpagominimo  Visa_mpagominimo  
fin drift_rank_cero_fijo()


In [ ]:

colnames(dataset)

[1] "numero_de_cliente"            "foto_mes"                    
 [3] "internet"                     "cliente_edad"                
 [5] "cliente_antiguedad"           "cproductos"                  
 [7] "cdescubierto_preacordado"     "ctarjeta_visa_transacciones" 
 [9] "cpayroll_trx"                 "ccomisiones_mantenimiento"   
[11] "ccallcenter_transacciones"    "chomebanking_transacciones"  
[13] "ctrx_quarter"                 "Master_status"               
[15] "Master_fechaalta"             "Visa_status"                 
[17] "Visa_fechaalta"               "clase_ternaria"              
[19] "mrentabilidad_rank"           "mrentabilidad_annual_rank"   
[21] "mcomisiones_rank"             "mactivos_margen_rank"        
[23] "mpasivos_margen_rank"         "mcuenta_corriente_rank"      
[25] "mcaja_ahorro_rank"            "mcuentas_saldo_rank"         
[27] "mtarjeta_visa_consumo_rank"   "mtarjeta_master_consumo_rank"
[29] "mprestamos_personales_rank"   "mpayroll_rank"               
[31] "Master_mpagominimo_rank"      "Visa_mpagominimo_rank"

#### 8.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [ ]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]


# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_por_edad := mpayroll * cliente_edad]


In [ ]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "numero_de_cliente"            "foto_mes"                    
 [3] "internet"                     "cliente_edad"                
 [5] "cliente_antiguedad"           "cproductos"                  
 [7] "cdescubierto_preacordado"     "ctarjeta_visa_transacciones" 
 [9] "cpayroll_trx"                 "ccomisiones_mantenimiento"   
[11] "ccallcenter_transacciones"    "chomebanking_transacciones"  
[13] "ctrx_quarter"                 "Master_status"               
[15] "Master_fechaalta"             "Visa_status"                 
[17] "Visa_fechaalta"               "clase_ternaria"              
[19] "mrentabilidad_rank"           "mrentabilidad_annual_rank"   
[21] "mcomisiones_rank"             "mactivos_margen_rank"        
[23] "mpasivos_margen_rank"         "mcuenta_corriente_rank"      
[25] "mcaja_ahorro_rank"            "mcuentas_saldo_rank"         
[27] "mtarjeta_visa_consumo_rank"   "mtarjeta_master_consumo_rank"
[29] "mprestamos_personales_rank"   "mpayroll_rank"               
[31] "Master_mpagominimo_rank"      "Visa_mpagominimo_rank"       
[33] "kmes"

#### 8.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [ ]:
# No se implementa Feature Engineering a partir de Random Forest

#### 8.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

*** Dejamos estos que son los que mejor resultado nos dieron en los experimentos colaborativos ***

In [ ]:
# Feature Engineering Historico
# ordeno el dataset
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [ ]:
ncol(dataset)
colnames(dataset)

[1] 153

[1] "numero_de_cliente"                  
  [2] "foto_mes"                           
  [3] "internet"                           
  [4] "cliente_edad"                       
  [5] "cliente_antiguedad"                 
  [6] "cproductos"                         
  [7] "cdescubierto_preacordado"           
  [8] "ctarjeta_visa_transacciones"        
  [9] "cpayroll_trx"                       
 [10] "ccomisiones_mantenimiento"          
 [11] "ccallcenter_transacciones"          
 [12] "chomebanking_transacciones"         
 [13] "ctrx_quarter"                       
 [14] "Master_status"                      
 [15] "Master_fechaalta"                   
 [16] "Visa_status"                        
 [17] "Visa_fechaalta"                     
 [18] "clase_ternaria"                     
 [19] "mrentabilidad_rank"                 
 [20] "mrentabilidad_annual_rank"          
 [21] "mcomisiones_rank"                   
 [22] "mactivos_margen_rank"               
 [23] "mpasivos_margen_rank"               
 [24] "mcuenta_corriente_rank"             
 [25] "mcaja_ahorro_rank"                  
 [26] "mcuentas_saldo_rank"                
 [27] "mtarjeta_visa_consumo_rank"         
 [28] "mtarjeta_master_consumo_rank"       
 [29] "mprestamos_personales_rank"         
 [30] "mpayroll_rank"                      
 [31] "Master_mpagominimo_rank"            
 [32] "Visa_mpagominimo_rank"              
 [33] "kmes"                               
 [34] "internet_lag1"                      
 [35] "cliente_edad_lag1"                  
 [36] "cliente_antiguedad_lag1"            
 [37] "cproductos_lag1"                    
 [38] "cdescubierto_preacordado_lag1"      
 [39] "ctarjeta_visa_transacciones_lag1"   
 [40] "cpayroll_trx_lag1"                  
 [41] "ccomisiones_mantenimiento_lag1"     
 [42] "ccallcenter_transacciones_lag1"     
 [43] "chomebanking_transacciones_lag1"    
 [44] "ctrx_quarter_lag1"                  
 [45] "Master_status_lag1"                 
 [46] "Master_fechaalta_lag1"              
 [47] "Visa_status_lag1"                   
 [48] "Visa_fechaalta_lag1"                
 [49] "mrentabilidad_rank_lag1"            
 [50] "mrentabilidad_annual_rank_lag1"     
 [51] "mcomisiones_rank_lag1"              
 [52] "mactivos_margen_rank_lag1"          
 [53] "mpasivos_margen_rank_lag1"          
 [54] "mcuenta_corriente_rank_lag1"        
 [55] "mcaja_ahorro_rank_lag1"             
 [56] "mcuentas_saldo_rank_lag1"           
 [57] "mtarjeta_visa_consumo_rank_lag1"    
 [58] "mtarjeta_master_consumo_rank_lag1"  
 [59] "mprestamos_personales_rank_lag1"    
 [60] "mpayroll_rank_lag1"                 
 [61] "Master_mpagominimo_rank_lag1"       
 [62] "Visa_mpagominimo_rank_lag1"         
 [63] "kmes_lag1"                          
 [64] "internet_lag2"                      
 [65] "cliente_edad_lag2"                  
 [66] "cliente_antiguedad_lag2"            
 [67] "cproductos_lag2"                    
 [68] "cdescubierto_preacordado_lag2"      
 [69] "ctarjeta_visa_transacciones_lag2"   
 [70] "cpayroll_trx_lag2"                  
 [71] "ccomisiones_mantenimiento_lag2"     
 [72] "ccallcenter_transacciones_lag2"     
 [73] "chomebanking_transacciones_lag2"    
 [74] "ctrx_quarter_lag2"                  
 [75] "Master_status_lag2"                 
 [76] "Master_fechaalta_lag2"              
 [77] "Visa_status_lag2"                   
 [78] "Visa_fechaalta_lag2"                
 [79] "mrentabilidad_rank_lag2"            
 [80] "mrentabilidad_annual_rank_lag2"     
 [81] "mcomisiones_rank_lag2"              
 [82] "mactivos_margen_rank_lag2"          
 [83] "mpasivos_margen_rank_lag2"          
 [84] "mcuenta_corriente_rank_lag2"        
 [85] "mcaja_ahorro_rank_lag2"             
 [86] "mcuentas_saldo_rank_lag2"           
 [87] "mtarjeta_visa_consumo_rank_lag2"    
 [88] "mtarjeta_master_consumo_rank_lag2"  
 [89] "mprestamos_personales_rank_lag2"    
 [90] "mpayroll_rank_lag2"                 
 [91] "Master_mpagominimo_rank_lag2"      

#### 8.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

*** No incluimos canaritos porque no aportaron significativamente en los experimentos colaborativos ***

In [ ]:
# No se implementa la reduccion de la dimensionalidad con canaritos
# No lo incluyo porque no mejora el rendimiento economico del modelo

### 8.3.2 Modelado

#### 8.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  con un undersampling de los CONTINUA del 2% por un tema de velocidad

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [ ]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [ ]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [ ]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 16551

####  8.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [ ]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 10

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.03,
  feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L),
  makeNumericParam("coverage", lower = 0.05, upper = 1.0), # nuevo
  makeNumericParam("min_gain_to_split", lower = 0.0, upper = 10.0),
  makeNumericParam("bagging_fraction", lower = 0.5, upper = 1.0)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [ ]:

# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  message(format(Sys.time(), "%a %b %d %X %Y"))

  # uno la lista de hiperparametros : fijos + variables
  param_completo <- c(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [ ]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [ ]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Sun Jul 20 11:00:48 PM 2025

Sun Jul 20 11:02:02 PM 2025

Sun Jul 20 11:03:46 PM 2025

Sun Jul 20 11:05:08 PM 2025

Sun Jul 20 11:06:32 PM 2025

Sun Jul 20 11:07:35 PM 2025

Sun Jul 20 11:08:44 PM 2025

Sun Jul 20 11:09:49 PM 2025

Sun Jul 20 11:10:50 PM 2025

Sun Jul 20 11:11:48 PM 2025

Sun Jul 20 11:12:53 PM 2025

Sun Jul 20 11:14:04 PM 2025

Sun Jul 20 11:15:12 PM 2025

Sun Jul 20 11:16:04 PM 2025

Sun Jul 20 11:17:10 PM 2025

Sun Jul 20 11:18:17 PM 2025

Sun Jul 20 11:19:14 PM 2025

Sun Jul 20 11:20:29 PM 2025

Sun Jul 20 11:21:35 PM 2025

Sun Jul 20 11:22:30 PM 2025

[mbo] 0: num_leaves=128; min_data_in_leaf=6575; coverage=0.918; min_gain_to_split=3.46; bagging_fraction=0.774 : y = 0.923 : 73.8 secs : initdesign

[mbo] 0: num_leaves=81; min_data_in_leaf=5513; coverage=0.568; min_gain_to_split=0.67; bagging_fraction=0.745 : y = 0.927 : 104.7 secs : initdesign

[mbo] 0: num_leaves=137; min_data_in_leaf=4598; coverage=0.673; min_gain_

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(num_leaves, min_data_in_leaf, num_iterations)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:        160               17            185


### 8.3.3 Produccion Semillerio

### Final Training Semillerio
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [ ]:
PARAM$trainingstrategy$final_train <- c( 202107,
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

[1] 240562

#### Final Training Hyperparameters

In [ ]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

#### Training  Semillerio
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

### Aqui va cuantas semillas utilizo para el semillerio

In [ ]:
# semillerio
PARAM$FT$semillerio <- 50  # cantidad de semillas

In [ ]:
# semillerio
if(!require("primes")) install.packages("primes")
require("primes")

In [ ]:
# semillerio
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$FT$semillas <- sample(primos)[seq(PARAM$FT$semillerio)]

cat( PARAM$FT$semillas)

586577 726809 298937 617467 268547 536891 344017 520631 377477 583139 750797 411589 612193 187633 849601 208591 622669 807299 325319 509741 696737 236869 766907 208889 312623 579539 233693 160687 630181 414283 927709 905143 395173 715439 245911 126341 540101 133811 385129 675151 951413 472063 374447 424121 469717 471407 216233 857321 128351 776879

In [ ]:
# semillerio
dir.create("modelos", showWarnings =FALSE)

primero <- TRUE
for( sem in PARAM$FT$semillas)
{
  nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
  if( !file.exists(nombre_arch) )
  {
    param_final$seed <- sem

    set.seed(sem, kind = "L'Ecuyer-CMRG")
    final_model <- lgb.train(
      data= dfinal_train,
      param= param_final,
      verbose= -100
    )

    lgb.save(final_model, nombre_arch) # grabo el modelo"

    # grabo la primer importancia de variables
    #  Natalia : da lo mismo cual se guarda
    if( primero)
    {
      primero <- FALSE
      tb_importancia <- as.data.table(lgb.importance(final_model))
      archivo_importancia <- "impo.txt"

      fwrite( tb_importancia,
        file= archivo_importancia,
        sep= "\t"
      )
    }
 }
}

### Scoring Semillerio

Aplico el modelo final a los datos del futuro

In [ ]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [ ]:
# semillerio aplico final_model   a dfuture

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := 0]

datos_matrix <- data.matrix(dfuture[, campos_buenos, with= FALSE])


for( isem in seq(length(PARAM$FT$semillas)) )
{
   sem <- PARAM$FT$semillas[ isem ]
   nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
   final_model <- lgb.load(nombre_arch)

   prediccion <- predict(
     final_model,
     datos_matrix
  )

  tb_prediccion[, paste0("prob_", isem) := prediccion]
  tb_prediccion[, prob := prob + prediccion]

  rm(final_model)
  rm(prediccion)
  gc(full = TRUE, verbose=FALSE)
}

rm( datos_matrix)
gc(full = TRUE, verbose=FALSE)

tb_prediccion[, prob := prob /length(PARAM$FT$semillas) ]

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2467374,131.8,5960914,318.4,5960914,318.4
Vcells,41131611,313.9,118933912,907.4,118885888,907.1


In [ ]:
tb_prediccion

numero_de_cliente,prob,prob_1,prob_2,prob_3,prob_4,prob_5,prob_6,prob_7,prob_8,⋯,prob_41,prob_42,prob_43,prob_44,prob_45,prob_46,prob_47,prob_48,prob_49,prob_50
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29186441,0.0005138773,0.0004319605,0.0005686816,0.0004517804,5.617916e-04,5.461566e-04,0.0005830474,4.231757e-04,0.0006717540,⋯,0.0008769628,0.0004974973,0.0007793240,0.0005206655,0.0005596259,3.721484e-04,0.0004360431,0.0003555122,0.0005033984,0.0004858121
29187961,0.0007936838,0.0010266699,0.0007640490,0.0011441499,9.816924e-04,9.618457e-04,0.0003797630,1.341818e-03,0.0006467255,⋯,0.0011077601,0.0008876656,0.0006115217,0.0005443717,0.0005189626,8.436962e-04,0.0007779737,0.0011416763,0.0007879558,0.0010399132
29193101,0.0004371930,0.0003796335,0.0004917827,0.0003876199,3.173636e-04,5.676184e-04,0.0003467465,5.861722e-04,0.0003542999,⋯,0.0003361426,0.0002849330,0.0003937918,0.0004119309,0.0003756132,3.488625e-04,0.0003789991,0.0004582496,0.0003766297,0.0005788563
29193281,0.0003492178,0.0004353794,0.0003330004,0.0003457959,3.508315e-04,3.533858e-04,0.0003009704,3.007872e-04,0.0002887257,⋯,0.0003771999,0.0002989981,0.0004768291,0.0002603318,0.0002797371,3.175258e-04,0.0004397135,0.0002663566,0.0002520017,0.0003214293
29198891,0.0004335470,0.0004155354,0.0004502287,0.0004326422,4.416154e-04,3.608536e-04,0.0003742916,4.441525e-04,0.0003477035,⋯,0.0005215381,0.0003595774,0.0004560716,0.0003930842,0.0003991746,4.209338e-04,0.0005373625,0.0003650355,0.0004299279,0.0004170045
29200651,0.0017398559,0.0017044752,0.0022018716,0.0015281301,8.434641e-04,1.976325e-03,0.0010035484,2.226445e-03,0.0016741152,⋯,0.0010045562,0.0022315798,0.0016046256,0.0024424095,0.0015322283,1.417682e-03,0.0010870573,0.0029874345,0.0018672005,0.0023254570
29201701,0.0002612258,0.0002147773,0.0002852023,0.0002201873,3.795719e-04,2.386459e-04,0.0002156018,2.561901e-04,0.0002399528,⋯,0.0002633000,0.0003843978,0.0002413238,0.0002796067,0.0003028835,2.617116e-04,0.0003034301,0.0002743067,0.0002980399,0.0001893705
29205441,0.0905308479,0.3791595850,0.0420129915,0.0304755667,3.542014e-02,4.092389e-02,0.1179530638,9.077073e-02,0.0519920633,⋯,0.1159848376,0.1400154409,0.0190457177,0.0278043641,0.1018142835,1.044651e-01,0.1480655208,0.0499271181,0.1373855628,0.0474812894
29207901,0.0029996031,0.0033491622,0.0024042618,0.0021478514,4.112310e-03,3.850658e-03,0.0027563199,3.370457e-03,0.0023853488,⋯,0.0025578771,0.0028053792,0.0029219273,0.0022763776,0.0023114892,2.595299e-03,0.0023557894,0.0026194867,0.0029792125,0.0025311639


In [ ]:
# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos

fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "labo-i-2025-rosario-analista-jr"   o  la original "labo-i-2025-rosario"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labo-i-2025-rosario-gerencial"
PARAM$kaggle$cortes <- seq(1050, 1200, by = 10)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0(
    "./kaggle/KAS",
    PARAM$experimento, "_",
    "s", length(PARAM$FT$semillas), "_",
    envios, ".csv"
  )

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
  "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 
Successfully submitted to LaboI 2025 Rosario Gerencial 


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-rosario-gerencial -f ./kaggle/KASfinala_s50_1190.csv -m 'envios=1190  semilla=888877'' had status 1”


429 Client Error: Too Many Requests for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/labo-i-2025-rosario-gerencial 


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-rosario-gerencial -f ./kaggle/KASfinala_s50_1200.csv -m 'envios=1200  semilla=888877'' had status 1”


429 Client Error: Too Many Requests for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/labo-i-2025-rosario-gerencial 


In [ ]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Jul 21 12:15:24 AM 2025"